[View in Colaboratory](https://colab.research.google.com/github/Mashiku7/Control-Anopheles-Malaria-Project-CAMP/blob/master/CAMP_HackMIT.ipynb)

Time series forecasting  maleria prevelance in Muleba using ARIMA

In [0]:
from __future__ import print_function

import math

from IPython import display
from matplotlib import cm
from matplotlib import gridspec
from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
from sklearn import metrics
import tensorflow as tf
from tensorflow.python.data import Dataset

tf.logging.set_verbosity(tf.logging.ERROR)
pd.options.display.max_rows = 10
pd.options.display.float_format = '{:.1f}'.format

In [0]:
data = pd.read_csv(“CDO6199837732377EnviromentDataForMIT - CDO6199837732377EnviromentData”)

Examine Data

In [0]:
data.describe()